In [1]:
import torch
import wandb
import diffusion_pde as dpde
from pathlib import Path
from hydra import initialize, compose
from omegaconf import OmegaConf

In [5]:
data_path = Path("/home/s204790/dynamical-pde-diffusion/data/llg_no_time.hdf5")
data_path_test = Path("/home/s204790/dynamical-pde-diffusion/data/llg_no_time_test.hdf5")
print(data_path.exists(), data_path_test.exists())
data_path.stem

True True


'llg_no_time'

In [ ]:
with initialize(config_path="../conf", version_base=None):
    cfg = compose(config_name="train", overrides=[])

wandb_kwargs = OmegaConf.to_container(cfg.wandb, resolve=True)
wandb_kwargs["name"] = f"test-datasets/dataset"
wandb_kwargs

{'entity': 'philiphohwy-danmarks-tekniske-universitet-dtu',
 'project': 'dynamical-pde-diffusion-final',
 'dir': '/home/s204790/logs',
 'name': 'heat-test-datasets/dataset',
 'mode': 'online'}

In [9]:
data_dir = dpde.utils.get_repo_root() / "data"

files = [p for p in data_dir.iterdir() if p.is_file() and "test" in p.stem]

[f.stem for f in files]

['heat_lineart_test',
 'llg_no_time_test',
 'heat_logt_test',
 'llg_test',
 'heat_no_time_test',
 'heat_no_cond_test',
 'heat_no_cond_test2']

In [10]:
with wandb.init(**wandb_kwargs) as run:
    artifact = wandb.Artifact(name="all-test-datasets", type="dataset")
    for file_path in files:
        artifact.add_file(str(file_path))
    artifact.metadata = {
        "description": "all test datasets for LLG and heat equation.",
    }
    run.log_artifact(artifact)

In [ ]:
api = wandb.Api()